# Isochrone using HERE API

In [31]:
# Import Libraries
from here.platform import Platform
from here.geopandas_adapter import GeoPandasAdapter
import flexpolyline as fp
import geopandas as gpd
import folium
from shapely.geometry import Polygon, mapping
import json
import os
import time

In [2]:
# Initialize Platform
platform = Platform(adapter=GeoPandasAdapter())

In [3]:
# Check to see if isoline services is available in our account
services = platform.list_services()

for service in services:
    print(f"{service.name} ({service.hrn})")
    
services

HERE Isoline Routing (hrn:here:service::olp-here:routing-isoline-8)
HERE Routing (hrn:here:service::olp-here:routing-8)


In [4]:
# Enable isochrone and routing services
isoline = platform.get_service("hrn:here:service::olp-here:routing-isoline-8")
routing = platform.get_service('hrn:here:service::olp-here:routing-8')

In [5]:
# Function to check whether isochrone data already exist or not. If not, hit the API. The function aims to save on the limited API usage

def get_isochrone_data(isochrone_parameter):
    # Check if isochrone data exist
    if os.path.exists('isochrone_result.json'):
        with open('isochrone_result.json', 'r') as f:
            isochrone_data = json.load(f)
            return isochrone_data
    else:
        # If it doesn't, hit API
        isochrone_data = isoline.get('/isolines', params = isochrone_parameter)
        
        # Save the result to API
        json_string = json.dumps(isochrone_data)
        json_file = open('isochrone_result.json', 'w')
        json_file.write(json_string)
        json_file.close()
        
        # Return the API result
        return isochrone_data

In [6]:
# Create isochrone parameter
params = {
    "origin": "-6.2363373,106.8010171", # Origin coordinate
    "range[type]": "time",
    "range[values]": "900,1800,3600", # Time interval
    "transportMode": "car",
    "optimizeFor": "quality", 
    "avoid[features]": "tollRoad",
    "traffic[mode]": "default", # Incorporate traffic data
    "departureTime": "2023-09-22T18:30:00"
}

In [7]:
# Run the function
isochrone_data = get_isochrone_data(params)

In [8]:
# Decode the flexible polygon
def decode_polygon(data):
    for time_interval in data['isolines']:
        time_interval['decoded_polygon'] = []
        decode = fp.decode(time_interval['polygons'][0]['outer'])
        time_interval['decoded_polygon'].append(decode)
    return data

In [9]:
# Run decoding function
decoded_isochrone = decode_polygon(isochrone_data)

In [10]:
# Create geodataframes from the decoded isochrone polygon
travel_time = [
    '15 Minutes',
    '30 Minutes',
    '60 Minutes'
]

# Origin point coordinate
origin_point = (-6.2363373, 106.8010171)

# Create polygon
polygon_coord_list = []

for i in decoded_isochrone['isolines']:
    polygon_coord_list.append(
        Polygon(i['decoded_polygon'][0])
    )

# Convert polygon to geodataframe
gdf_isochrone = gpd.GeoDataFrame(
    {
        'name': travel_time,
        'geometry': polygon_coord_list,
    }
)

# Set datum
gdf_isochrone = gdf_isochrone.set_crs(4326)

# Clip overlapping areas
gdf_isochrone['geometry'][2] = gdf_isochrone['geometry'][2].difference(gdf_isochrone['geometry'][1])
gdf_isochrone['geometry'][1] = gdf_isochrone['geometry'][1].difference(gdf_isochrone['geometry'][0])

In [11]:
exterior_coords_list = []

for index, row in gdf_isochrone.iterrows():
    ext_coord_row = (row['geometry']).exterior.coords[:-1]
    exterior_coords_list.append(ext_coord_row)

exterior_coords_list

[[(-6.234741, 106.77372),
  (-6.234741, 106.776123),
  (-6.235428, 106.778183),
  (-6.236801, 106.779556),
  (-6.236801, 106.78093),
  (-6.234741, 106.781616),
  (-6.232681, 106.78093),
  (-6.231308, 106.779556),
  (-6.229248, 106.77887),
  (-6.227188, 106.779556),
  (-6.223068, 106.783676),
  (-6.218948, 106.785049),
  (-6.217575, 106.789169),
  (-6.212082, 106.794662),
  (-6.210022, 106.795349),
  (-6.207275, 106.795349),
  (-6.205215, 106.796036),
  (-6.203842, 106.797409),
  (-6.199722, 106.798782),
  (-6.199722, 106.800156),
  (-6.203842, 106.804276),
  (-6.204529, 106.806335),
  (-6.204529, 106.809082),
  (-6.203842, 106.811142),
  (-6.202469, 106.811142),
  (-6.201096, 106.809769),
  (-6.199722, 106.809769),
  (-6.196976, 106.812515),
  (-6.196289, 106.814575),
  (-6.196976, 106.816635),
  (-6.198349, 106.818008),
  (-6.198349, 106.819382),
  (-6.194229, 106.820755),
  (-6.193542, 106.822815),
  (-6.194229, 106.824875),
  (-6.196289, 106.825562),
  (-6.204529, 106.825562),
  (-6

In [36]:
tes_distance = [
    [
        (-6.234741, 106.77372),
        (-6.234741, 106.776123),
        (-6.235428, 106.778183)
    ],
    [
        (-6.236801, 106.779556),
        (-6.236801, 106.78093),
        (-6.234741, 106.781616)
    ],
    [
        (-6.232681, 106.78093),
        (-6.231308, 106.779556),
        (-6.229248, 106.77887)
    ]
]

travel_distance = []

for sublist in tes_distance:
    sublist_travel_distance = []

    for coord_pair in sublist:
        # Wait for one second before making the next request
        time.sleep(1)
        
        # Make request for the routing API
        response = routing.get(
                '/routes',
                params={
                    'transportMode': 'car',
                    'origin': str(origin_point[0]) + ',' + str(origin_point[1]),
                    'destination': str(coord_pair[0]) + ','+ str(coord_pair[1]),
                    'departureTime': '2023-09-22T18:30:00',
                    'avoid[features]': 'tollRoad',
                    'units': 'metric',
                    'return': 'summary',
                    'traffic[mode]': 'default'
                })
        
        # Get the length of each route
        route_length = response['routes'][0]['sections'][0]['summary']['length']

        # Add route length to each isochrone duration
        sublist_travel_distance.append(route_length)

    # Add all route length to travel_distance
    travel_distance.append(sublist_travel_distance)

In [37]:
travel_distance

[[4374, 4421, 4719], [4845, 4755, 5340], [5274, 5401, 5857]]

In [ ]:
# Get furthest distance
gdf_isochrone['furthest_point'] = gdf_isochrone['geometry'].apply(lambda x: furthest_point_in_polygon(origin_point, x))

In [ ]:
# Create map
map = folium.Map(
    location=origin_point,
    zoom_start=11,
    tiles='OpenStreetMap'
)

# Create marker for origion point
folium.Marker(
    location=[-6.2363373, 106.8010171],
    icon=folium.Icon('blue'),
    popup='Origin Point'
).add_to(map)

# Create feature groups of all isochrone duration
feature_groups = [folium.FeatureGroup(
    name=name,
    overlay=True
) for name in travel_time]

# Create polygons of isochrone
polygon_list = []
coord_pairs = []
color_list = ['green', 'yellow', 'red']

for i, row in gdf_isochrone.iterrows():
    # Extract polygon coordinates from shapely polygon to list of coodinate pairs
    coord_pairs.append(
        list(mapping(row['geometry'])['coordinates'])
    )

    # Create polygon for each isochrone duration
    polygon_list.append(
        folium.Polygon(
            locations= coord_pairs[i],
            fill_color= color_list[i],
            fill_opacity = 0.3,
            tooltip= row['name']
        ))

# Append each polygon to feature group    
for i in range(len(feature_groups)):
    polygon_list[i].add_to(feature_groups[i])
    feature_groups[i].add_to(map)

# Add layer control to toggle the layers
folium.LayerControl().add_to(map)

# Show the map
map